In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [4]:
df_claims = pd.read_csv('data/Claims.csv')
df_members = pd.read_csv('data/Members.csv')
df_hospital = pd.read_csv('data/DaysInHospital_Y2.csv')
df_drug = pd.read_csv('data/DrugCount.csv')
df_lab = pd.read_csv('data/LabCount.csv')

In [11]:
df_claims.dtypes

MemberID                   int64
ProviderID               float64
Vendor                   float64
PCP                      float64
Year                      object
Specialty                 object
PlaceSvc                  object
PayDelay                   int32
LengthOfStay               int32
DSFS                       int32
PrimaryConditionGroup     object
CharlsonIndex              int32
ProcedureGroup            object
SupLOS                     int64
dtype: object

In [12]:
df_claims

,MemberID,ProviderID,Vendor,PCP,Year,Specialty,PlaceSvc,PayDelay,LengthOfStay,DSFS,PrimaryConditionGroup,CharlsonIndex,ProcedureGroup,SupLOS
0,42286978,8013252.0,172193.0,37796.0,Y1,Surgery,Office,28,0,9,NEUMENT,0,MED,0
1,97903248,3316066.0,726296.0,5300.0,Y3,Internal,Office,50,0,8,NEUMENT,2,EM,0
2,2759427,2997752.0,140343.0,91972.0,Y3,Internal,Office,14,0,1,METAB3,0,EM,0
3,73570559,7053364.0,240043.0,70119.0,Y3,Laboratory,Independent Lab,24,0,6,METAB3,2,SCS,0
4,11837054,7557061.0,496247.0,68968.0,Y2,Surgery,Outpatient Hospital,27,0,5,FXDISLC,2,EM,0
5,45844561,1963488.0,4042.0,55823.0,Y3,Pediatrics,Office,25,0,4,NEUMENT,0,EM,0
6,99829076,6721023.0,265273.0,91972.0,Y1,Rehabilitation,Office,162,0,1,TRAUMA,0,MED,0
7,54666321,9932074.0,35565.0,27294.0,Y1,Diagnostic Imaging,Office,29,0,1,RESPR4,0,RAD,0
8,60497718,363858.0,293107.0,64913.0,Y2,Rehabilitation,Office,42,0,2,INFEC4,0,EM,0
9,72200595,6251259.0,791272.0,49465.0,Y3,Internal,Office,56,0,8,MISCHRT,2,SCS,0


In [5]:
def count_nulls(df):
    null_counter = df.isnull().sum(axis=0)
    null_counter = null_counter[null_counter > 0]
    null_percent = df.isnull().sum(axis=0) / df.shape[0] * 100
    null_percent = null_percent[null_percent > 0]
    null_df = pd.concat([null_counter,null_percent],axis=1)
    null_df.columns = ['count','percent']
    display(null_df)
    
count_nulls(df_claims)

,count,percent
ProviderID,16264,0.609369
Vendor,24856,0.931289
PCP,7492,0.280705
Specialty,8405,0.314913
PlaceSvc,7632,0.285951
LengthOfStay,2597392,97.317412
DSFS,52770,1.977152
PrimaryConditionGroup,11410,0.427503
ProcedureGroup,3675,0.137693


In [7]:
def updatePayDelay():
    df_claims.loc[df_claims['PayDelay']=='162+','PayDelay'] = 162
    df_claims['PayDelay'] = df_claims['PayDelay'].astype(int)
    return

updatePayDelay()

In [8]:
def updateDSFS():
    df_claims.loc[df_claims['DSFS']=='0- 1 month','DSFS'] = 1
    df_claims.loc[df_claims['DSFS']=='1- 2 months','DSFS'] = 2
    df_claims.loc[df_claims['DSFS']=='2- 3 months','DSFS'] = 3
    df_claims.loc[df_claims['DSFS']=='3- 4 months','DSFS'] = 4
    df_claims.loc[df_claims['DSFS']=='4- 5 months','DSFS'] = 5
    df_claims.loc[df_claims['DSFS']=='5- 6 months','DSFS'] = 6
    df_claims.loc[df_claims['DSFS']=='6- 7 months','DSFS'] = 7
    df_claims.loc[df_claims['DSFS']=='7- 8 months','DSFS'] = 8
    df_claims.loc[df_claims['DSFS']=='8- 9 months','DSFS'] = 9
    df_claims.loc[df_claims['DSFS']=='9-10 months','DSFS'] = 10
    df_claims.loc[df_claims['DSFS']=='10-11 months','DSFS'] = 11
    df_claims.loc[df_claims['DSFS']=='11-12 months','DSFS'] = 12
    df_claims['DSFS'].fillna(0, inplace=True)
    df_claims['DSFS'] = df_claims['DSFS'].astype(int)
    return

updateDSFS()

In [9]:
def updateCharlsonIndex():
    df_claims.loc[df_claims['CharlsonIndex']=='0','CharlsonIndex'] = 0
    df_claims.loc[df_claims['CharlsonIndex']=='1-2','CharlsonIndex'] = 2
    df_claims.loc[df_claims['CharlsonIndex']=='3-4','CharlsonIndex'] = 4
    df_claims.loc[df_claims['CharlsonIndex']=='5+','CharlsonIndex'] = 6

    df_claims['CharlsonIndex'] = df_claims['CharlsonIndex'].astype(int)
    return
    
updateCharlsonIndex()

In [10]:
def updateLengthOfStay():
    df_claims.loc[df_claims['LengthOfStay']=='1 day','LengthOfStay'] = 1
    df_claims.loc[df_claims['LengthOfStay']=='2 days','LengthOfStay'] = 2
    df_claims.loc[df_claims['LengthOfStay']=='3 days','LengthOfStay'] = 3
    df_claims.loc[df_claims['LengthOfStay']=='4 days','LengthOfStay'] = 4
    df_claims.loc[df_claims['LengthOfStay']=='5 days','LengthOfStay'] = 5
    df_claims.loc[df_claims['LengthOfStay']=='6 days','LengthOfStay'] = 6
    df_claims.loc[df_claims['LengthOfStay']=='1- 2 weeks','LengthOfStay'] = 11
    df_claims.loc[df_claims['LengthOfStay']=='2- 4 weeks','LengthOfStay'] = 21
    df_claims.loc[df_claims['LengthOfStay']=='4- 8 weeks','LengthOfStay'] = 42
    df_claims.loc[df_claims['LengthOfStay']=='26+ weeks','LengthOfStay'] = 180
    
    df_claims['LengthOfStay'].fillna(0, inplace=True)

    df_claims['LengthOfStay'] = df_claims['LengthOfStay'].astype(int)
    return

updateLengthOfStay()

In [16]:
df_claims['no_Claims'] = df_claims.groupby(['Year','MemberID'])['ProviderID'].transform('count')

df_claims['no_Providers'] = df_claims.groupby(['Year','MemberID'])['ProviderID'].transform('nunique')
df_claims['no_Vendors'] = df_claims.groupby(['Year','MemberID'])['Vendor'].transform('nunique')
df_claims['no_PCPs'] = df_claims.groupby(['Year','MemberID'])['PCP'].transform('nunique')
df_claims['no_PlaceSvcs'] = df_claims.groupby(['Year','MemberID'])['PlaceSvc'].transform('nunique')
df_claims['no_Specialities'] = df_claims.groupby(['Year','MemberID'])['Specialty'].transform('nunique')
df_claims['no_PrimaryConditionGroups'] = df_claims.groupby(['Year','MemberID'])['PrimaryConditionGroup'].transform('nunique')
df_claims['no_ProcedureGroups'] = df_claims.groupby(['Year','MemberID'])['ProcedureGroup'].transform('nunique')

df_claims['PayDelay_max'] = df_claims.groupby(['Year','MemberID'])['PayDelay'].transform('max')
df_claims['PayDelay_min'] = df_claims.groupby(['Year','MemberID'])['PayDelay'].transform('min')
df_claims['PayDelay_ave'] = df_claims.groupby(['Year','MemberID'])['PayDelay'].transform(lambda x: np.average(x))
# df_claims['PayDelay_stdev'] = df_claims.groupby(['Year','MemberID'])['PayDelay'].transform(lambda x: 1 if x==0 else 0)
          
# df_claims['no_Vendors'] = df_claims.groupby(['Year','MemberID'])['Vendor'].transform('nunique')
# df_claims['no_Vendors'] = df_claims.groupby(['Year','MemberID'])['Vendor'].transform('nunique')
# df_claims['no_Vendors'] = df_claims.groupby(['Year','MemberID'])['Vendor'].transform('nunique')


In [14]:
df_claims

,ProviderID,Vendor,PCP,Specialty,PlaceSvc,PayDelay,LengthOfStay,DSFS,PrimaryConditionGroup,CharlsonIndex,ProcedureGroup,SupLOS
0,12,12,12,11,12,12,12,12,12,12,12,12
1,44,44,44,44,44,44,44,44,44,44,44,44
2,3,3,3,3,3,3,3,3,3,3,3,3
3,30,30,30,30,30,30,30,30,30,30,30,30
4,43,43,43,43,43,43,43,43,43,43,43,43
5,6,6,6,6,6,6,6,6,6,6,6,6
6,12,12,12,12,12,12,12,12,12,12,12,12
7,36,36,36,36,36,36,36,36,36,36,36,36
8,15,15,15,15,15,15,15,15,15,15,15,15
9,44,44,44,44,44,44,44,44,44,44,44,44
